In [2]:
import pandas as pd
import networkx as nx
import matplotlib
%matplotlib notebook
import matplotlib.pyplot as plt
from pprint import pprint
import seaborn as sns
import numpy as np
from collections import Counter

# Getting familiar with the NetworkX python module

### Building a network (or graph)

In [3]:
''## Undirected graph
Gud = nx.Graph()
## Add nodes
Gud.add_node(0)
Gud.add_node(1)
## Add links / edges
Gud.add_edge(0,1)
nx.draw_networkx(Gud)

<IPython.core.display.Javascript object>

In [4]:
## Directed graph
Gdir = nx.DiGraph()
Gdir.add_edge(0,1)
nx.draw_networkx(Gdir)

### Visualize the relationships with the adjacency matrix

In [5]:
Aud = nx.to_numpy_array(Gud)
plt.imshow(Aud,cmap="Greys")
plt.xticks(range(2))
plt.yticks(range(2))
plt.show()

In [6]:
Adir = nx.to_numpy_array(Gdir)
plt.imshow(Adir,cmap="Greys")
plt.xticks(range(2))
plt.yticks(range(2))
plt.show()

### Node and edge attributes 

In [7]:
Gud.nodes[0]["color"] = "red"
Gud.nodes(data=True)

NodeDataView({0: {'color': 'red'}, 1: {}})

In [8]:
Gud.add_node(2,color="blue")
Gud.nodes(data=True)

NodeDataView({0: {'color': 'red'}, 1: {}, 2: {'color': 'blue'}})

In [9]:
nx.draw_networkx(Gud)

In [10]:
Gud[0][1]["kind"] = "family"
Gud[0][1]["weight"] = 3
Gud.edges(data=True)

EdgeDataView([(0, 1, {'kind': 'family', 'weight': 3})])

# Building a graph from data

Dataset - **contacts** and **metadata** (Thiers13): https://zenodo.org/record/2540795

Description of dataset: http://www.sociopatterns.org/datasets/high-school-contact-and-friendship-networks/

Paper: https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0136497

Contacts of the students of nine classes during 5 days in Dec. 2013, as measured by the SocioPatterns infrastructure. The file contains a tab-separated list representing the active contacts during 20-second intervals of the data collection. Each line has the form “t i j“, where i and j are the anonymous IDs of the persons in contact and the interval during which this contact was active is [ t – 20s, t ]. If multiple contacts are active in a given interval, you will see multiple lines starting with the same value of t. Time is measured in seconds and expressed in UNIX ctime.

Sensors are embedded in unobtrusive wearable badges and exchange ultra-low power radio packets in order to detect close proximity of individuals wearing them. Students were asked to wear the sensors on their chests using lanyards, ensuring that the devices of two individuals can only exchange radio packets when the persons are facing each other. 

The classes have different specialization: “MP” classes focus more on mathematics and physics, “PC” classes on physics and chemistry, “PSI” classes on engineering studies and “BIO” classes on biology. We collected data among students of nine classes corresponding to the second year of such studies: 3 classes of type “MP” (MP, MPst1, MPst2), two of type “PC” (PC and PCst), one of type “PSI” (PSIst) and 3 of type “BIO” (2BIO1, 2BIO2, 2BIO3). 

### Load data to dataframes

In [11]:
contact = pd.read_csv("../Data/contact/tij_Thiers13.dat",names=['t', 'n1', 'n2'],delimiter=" ")
metadata = pd.read_csv("../Data/metadata/metadata_Thiers13.dat",names=["n","group"],delimiter="\t")

FileNotFoundError: [Errno 2] No such file or directory: '../Data/contact/tij_Thiers13.dat'

In [12]:
metadata.head()

NameError: name 'metadata' is not defined

In [13]:
classes = metadata['group'].unique()
print(classes)
class_to_int = {ci:i for i,ci in enumerate(classes)}
class_to_int

NameError: name 'metadata' is not defined

In [14]:
contact.head()

NameError: name 'contact' is not defined

In [15]:
## There are repeated interactions!
contact.loc[(contact["n1"]==454) & (contact["n2"]==640)]

NameError: name 'contact' is not defined

In [16]:
len(contact.loc[(contact["n1"]==454) & (contact["n2"]==640)])

NameError: name 'contact' is not defined

### Aggregate number of interactions

In [17]:
## Aggregate contacts
## https://stackoverflow.com/questions/38933071/group-by-two-columns-and-count-the-occurrences-of-each-combination-in-pandas
count_series = contact.groupby(['n1', 'n2']).size()
contact_aggr = count_series.to_frame(name = 'interactions').reset_index()
contact_aggr.head()

NameError: name 'contact' is not defined

In [18]:
contact_aggr.loc[(contact_aggr["n1"]==454) & (contact_aggr["n2"]==640)]

NameError: name 'contact_aggr' is not defined

In [19]:
contact_aggr["interactions_inv"] = contact_aggr["interactions"].map(lambda x: 1/x)
contact_aggr.head()

NameError: name 'contact_aggr' is not defined

### Distribution of number of interactions

In [20]:
## "Default" method
contact_aggr["interactions"].hist(bins=20)
plt.xlabel("Interactions")
plt.ylabel("Frequency")
plt.gca().grid(False)
plt.show()

NameError: name 'contact_aggr' is not defined

In [21]:
## Method for heterogeneous distributions (power laws)
intearctions_cntr = Counter(contact_aggr["interactions"])
interactions_xy = dict(intearctions_cntr).items()
x, y = zip(*interactions_xy)
plt.loglog(x,y,"o")
plt.xlabel("Interactions")
plt.ylabel("Frequency")
plt.show()

NameError: name 'contact_aggr' is not defined

### Build the network

In [22]:
## This is not useful for lists of undirected networks with repeated edge
## (some edges may appear as a,b and also as b,a)
## G = nx.from_pandas_edgelist(contact_aggr, "n1", "n2", edge_attr=["interactions","interactions_inv"], create_using=nx.Graph())

In [23]:
## Create undirected network
G = nx.Graph()
for ind, row in contact_aggr.iterrows():
    n1 = int(row["n1"])
    n2 = int(row["n2"])
    if not G.has_node(n1):
        group = metadata.loc[metadata['n'] == n1]["group"].values[0]
        G.add_node(n1,group=group)
    if not G.has_node(n2):
        group = metadata.loc[metadata['n'] == n2]["group"].values[0]
        G.add_node(n2,group=group)
    intr = row["interactions"]
    intr_inv = row["interactions_inv"]
    if G.has_edge(n1,n2):
        G[n1][n2]["interactions"] += intr
        G[n1][n2]["interactions_inv"] = 1/G[n1][n2]["interactions"]
    else:
        G.add_edge(n1,n2,interactions=intr,interactions_inv=intr_inv)

NameError: name 'contact_aggr' is not defined

### Draw the network

In [ ]:
## Compute a layout
pos = nx.kamada_kawai_layout(G,
                            weight="interactions",
                            scale=1)

In [ ]:
## Fix a node ordering
nodelist = G.nodes
## Convert classes labels to numbers for plotting
colors = [class_to_int[G.nodes[ni]["group"]] for ni in nodelist]
nx.draw_networkx(G,
                 pos=pos,
                 node_size=50,
                 font_size=7,
                 width=0.1, ## Edge width
                 nodelist=nodelist, 
                 node_color=colors,
                 cmap="Set1",
                 with_labels=True,
                 labels={ni:G.nodes[ni]["group"] for ni in nodelist}, ## Label each node (student) with the class she attends
                 font_color="k"
                )
plt.axis("off")

# Analyzing the network

### Average degree and density

In [ ]:
E = G.number_of_edges()
N = G.order()

In [ ]:
print ("Average degree=", 2*E/N)

In [ ]:
dens = E/(0.5*N*(N-1))
"Density", dens, nx.density(G)

## Node properties

### Degree centrality

In [ ]:
## Compute degree of the nodes
degree_dct = G.degree()
## Save it as node attribute in the graph object
nx.set_node_attributes(G, dict(degree_dct), name="degree")
## Compute weighted degree of the nodes
degree_w_dct = G.degree(weight="interactions")
## Save it as node attribute in the graph object
nx.set_node_attributes(G, dict(degree_w_dct), name="degree_w")

In [ ]:
G.nodes[1]

In [ ]:
# nodes_df = pd.DataFrame.from_dict(dict(degree_dct),orient="index",columns=["degree"])
metadata['degree'] = metadata["n"].map(dict(degree_dct))

In [ ]:
metadata.sort_values("degree",ascending=False).head(10)

In [ ]:
metadata['degree_w'] = metadata["n"].map(dict(degree_w_dct))
metadata.sort_values("degree_w",ascending=False).head(10)

### Degree distribution

In [ ]:
def discrete_distribution(x):
    cntr = Counter(x)
    distrib_xy = dict(cntr).items()
    x,y = zip(*distrib_xy)
    return x,y

In [ ]:
# from scipy import stats

In [ ]:
metadata["degree"].hist(bins=20,density=True)
# plt.plot(np.arange(0,100),stats.poisson.pmf(np.arange(0,100),2*E/N))
plt.xlabel("Degree")
plt.ylabel("Probability")
plt.gca().grid(False)
plt.show()

In [ ]:
metadata["degree_w"].hist(bins=30,density=True)
# plt.plot(np.arange(0,5000),stats.expon.pdf(np.arange(0,5000),scale=np.mean(metadata["degree_w"])))
plt.xlabel("Weight")
plt.ylabel("Probability")
plt.gca().grid(False)
plt.show()

### Draw network scaling node size with degree

In [ ]:
def rescale(x,xmin,xmax,a,b):
    s = (b-a)/(xmax-xmin)
    return (x-xmin)*s + a

In [ ]:
nodelist = G.nodes
node_size = [rescale(degree_w_dct[ni],1,max(metadata["degree_w"]),2,100) for ni in nodelist]
nx.draw_networkx(G,
                 pos=pos,
#                  node_size=100,
                 font_size=7,
                 width=0.1,
                 node_size=node_size,
                node_color=node_size)
plt.axis("off")

### Eigenvector centrality

In [ ]:
## Careful with the weight parameter!
eigen_dct = nx.eigenvector_centrality(G,weight="interactions",max_iter=1000)

In [ ]:
nx.set_node_attributes(G,eigen_dct,name="eigenvector")
metadata['eigenvector'] = metadata.n.map(eigen_dct)

In [ ]:
metadata.sort_values("eigenvector",ascending=False).head(10)

### PageRank centrality

In [ ]:
## Careful with the weight parameter!
pr_dct = nx.pagerank(G,weight="interactions",max_iter=1000)

In [ ]:
nx.set_node_attributes(G,pr_dct,name="pagerank")
metadata['pagerank'] = metadata.n.map(pr_dct)

In [ ]:
metadata.sort_values("pagerank",ascending=False).head(10)

### Betweenness centrality

In [ ]:
## Careful with the weight parameter! 
betwn_dct = nx.betweenness_centrality(G,weight="interactions_inv")

In [ ]:
nx.set_node_attributes(G,betwn_dct,name="betweenness")

In [ ]:
metadata['betweenness'] = metadata.n.map(betwn_dct)

In [ ]:
metadata.sort_values("betweenness",ascending=False).head(10)

### Closeness centrality

In [ ]:
## Careful with the weight parameter!
close_dct = nx.closeness_centrality(G,distance="interactions_inv")

In [ ]:
nx.set_node_attributes(G,close_dct,name="closeness")

In [ ]:
metadata['closeness'] = metadata.n.map(close_dct)

In [ ]:
metadata.sort_values("closeness",ascending=False).head(10)

### Clustering

In [ ]:
## Clustering
clust_dct = nx.clustering(G)

In [ ]:
nx.set_node_attributes(G,clust_dct,name="clustering")
metadata['clustering'] = metadata.n.map(clust_dct)

In [ ]:
metadata.sort_values("clustering",ascending=False).head(10)

## Correlation between different node properties

In [ ]:
plt.ioff()
## Pairplot
pg = sns.pairplot(metadata[["degree","degree_w","eigenvector","pagerank","betweenness","closeness","clustering"]])
pg.set(xscale="log",yscale="log")
plt.savefig("../Figures/node_prop_correl.pdf")
plt.close()
plt.ion()

# Groups of nodes (communities)

In [ ]:
import graph_tool
from graph_tool import draw
from graph_tool import inference

### Convert NetworkX to graph-tool

In [ ]:
## Networkx to graph-tool through adjacency matrix
nodelist = list(G.nodes)
A = nx.to_numpy_array(G,weight="interactions",nodelist=nodelist)
## Since the network is undirected, convert full matrix to upper triangular
Atu = np.triu(A)
## Extract non-zero indices (the list of links)
Anz = np.nonzero(Atu)

In [ ]:
## Build the network as per https://graph-tool.skewed.de/static/doc/quickstart.html (search for undirected graphs)
ug = graph_tool.Graph(
    np.array([Anz[0], Anz[1], np.log(Atu[Anz])]).T, ## List of edges with their weights
    eprops=[("weight", "double")], ## Creating the edge property to store the weight
    directed=False
    )

### Find communities by fitting Assortative Schocastic Blockmodel with graph-tool

In [ ]:
## Unweighted assortative
state_uw_ass = inference.minimize_blockmodel_dl(
    ug, 
    state=inference.PPBlockState ## Assortative model
    )
## Partition quality metric ("goodness of fit")- the lower the better
state_uw_ass.entropy()

In [ ]:
## Refine fit
for i in range(100):
    state_uw_ass.multiflip_mcmc_sweep(beta=np.inf, niter=10)
state_uw_ass.entropy()

In [ ]:
Counter(state_uw_ass.get_blocks())

In [ ]:
## Store result in our networkx graph
for i, n in enumerate(nodelist):
    r = state_uw_ass.get_blocks()[i]
    G.nodes[n]["assort_comm"] = r

In [ ]:
## Calculate node positions to plot result
pos = draw.sfdp_layout(ug,eweight=ug.ep.weight)

In [ ]:
## Plot result
state_uw_ass.draw(pos=pos, output="../Figures/thiers_uw_ass.png",output_size=(4096,4096))

### Hierarchical Schocastic Blockmodel (unweighted)

In [ ]:
state_hierachy = inference.minimize_nested_blockmodel_dl(ug)
state_hierachy.entropy()

In [ ]:
for i in range(100):
    state_hierachy.multiflip_mcmc_sweep(beta=np.inf, niter=10)

In [ ]:
state_hierachy.entropy()

In [ ]:
## Plot result
state_hierachy.draw(pos=pos, output="../Figures/Thiers_uw_hierarchical.png",output_size=(4048,4048))

In [ ]:
## Layers and blocks per layer
state_hierachy.print_summary()

In [ ]:
## Store result in networkx graph
levels_hierarchical = state_hierachy.get_levels()
for i, n in enumerate(nodelist):
    r = levels_hierarchical[0].get_blocks()[i]
    G.nodes[n]["lvl_0"] = r
    for lvli in range(1,10):
        r = levels_hierarchical[lvli].get_blocks()[r]
        G.nodes[n]["lvl_"+str(lvli)] = r

### Hierarchical Stochastic Blockmodel with edge weights

In [ ]:
np.log(contact_aggr["interactions"]).hist(bins=100,density=True)
plt.gca().grid(False)
plt.xlabel("log(Interactions)")
plt.ylabel("Probability")
# plt.semilogy()

In [ ]:
## Compute
state_wght = inference.minimize_nested_blockmodel_dl(
    ug,
    state_args=dict(recs=[ug.ep.weight],rec_types=["real-exponential"]) ## Model weights
    )

In [ ]:
state_wght.entropy()

In [ ]:
for i in range(100):
    state_wght.multiflip_mcmc_sweep(niter=10, beta=np.inf)

In [ ]:
state_wght.entropy()

In [ ]:
## Plot result
state_wght.draw(edge_color=ug.ep.weight, ecmap=(matplotlib.cm.inferno, .6),
           eorder=ug.ep.weight, edge_pen_width=draw.prop_to_size(ug.ep.weight, 2, 8, power=1),
           edge_gradient=[], output="../Figures/Thiers_weighted_hierarchical.pdf")

In [ ]:
## Layers and blocks per layer
state_wght.print_summary()

In [ ]:
## Store result in networkx graph
levels_wght = state_wght.get_levels()
for i, n in enumerate(nodelist):
    r = levels_wght[0].get_blocks()[i]
    G.nodes[n]["w_lvl_0"] = r
    for lvli in range(1,10):
        r = levels_wght[lvli].get_blocks()[r]
        G.nodes[n]["w_lvl_"+str(lvli)] = r

# k-core decomposition (extraction of the network backbone)

In [ ]:
cores_dct = {}
for ki in range(1,40):
    node_lst_i = nx.k_core(G,ki).nodes
    cores_dct_i = {ni:ki for ni in node_lst_i}
    cores_dct.update(cores_dct_i)
    print (len(node_lst_i))
    if len(node_lst_i) <= 0:
        break

In [ ]:
nx.set_node_attributes(G, cores_dct, name="k-core")

# Export NetworkX graph to read it with gephi

In [ ]:
nx.write_gexf(G,"../Results/Thiers_processed_TEST2.gexf")

## Extra: Is the network small-world?
https://chih-ling-hsu.github.io/2020/05/15/Gnp

In [ ]:
## Average degree of the network
kav = 2*E/N

In [ ]:
## Network density (probability that any two nodes are connected)
## This is equal to the clustering of a random network
kav/(N-1)

In [ ]:
## Average clustering of the network (much larger than for the random network)
nx.average_clustering(G)

In [ ]:
## Average path length of a random network (approximation)
np.log(N)/np.log(kav)

In [ ]:
## Average path length of our network
nx.average_shortest_path_length(G)